In [1]:
!pip install ldap3

In [ ]:
!docker run -p 389:389 --rm --name test-ldap rroemhild/test-openldap

In [ ]:
# Simple demonstration of LDAP authentication in Python
# Uses the popular 'ldap3' library to connect and bind to an LDAP server
# Works with Active Directory, OpenLDAP, etc.
# WARNING: Educational only – handle credentials securely in production!

from ldap3 import Server, Connection, ALL, NTLM, SIMPLE
import getpass

def ldap_auth_simple(host, port=389, use_ssl=False, base_dn=None,
                    username=None, password=None):
    """
    Simple LDAP authentication (username in DN format, e.g., uid=user,ou=people,dc=example,dc=com)
    """
    print(f"\n=== Simple LDAP Auth Demo ({'LDAPS' if use_ssl else 'LDAP'}) ===")
    
    server = Server(host, port=port, use_ssl=use_ssl, get_info=ALL)
    
    # Prompt for credentials if not provided
    if not username:
        username = input("Enter username (full DN, e.g., uid=john,ou=people,dc=example,dc=com): ")
    if not password:
        password = getpass.getpass("Enter password: ")
    
    try:
        conn = Connection(server, user=username, password=password, authentication=SIMPLE)
        if conn.bind():
            print("✓ Authentication successful!")
            print(f"Bound as: {conn.user}")
            # Optional: Show some user attributes
            if base_dn:
                conn.search(base_dn, f'(objectClass=*)', attributes=['cn', 'mail', 'uid'])
                print("User entry:", conn.entries)
        else:
            print("✗ Authentication failed.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.unbind()

def ldap_auth_ad(domain, username=None, password=None):
    """
    Active Directory authentication using NTLM (common in Windows environments)
    Username format: DOMAIN\\user or user@domain.com
    """
    print("\n=== Active Directory (NTLM) Auth Demo ===")
    
    # AD typically uses port 389 (LDAP) or 636 (LDAPS)
    server = Server(domain, get_info=ALL)  # e.g., 'dc01.example.com' or 'example.com'
    
    if not username:
        username = input("Enter username (e.g., EXAMPLE\\john or john@example.com): ")
    if not password:
        password = getpass.getpass("Enter password: ")
    
    try:
        conn = Connection(server, user=username, password=password, authentication=NTLM)
        if conn.bind():
            print("✓ AD Authentication successful!")
            print(f"Bound as: {conn.user}")
        else:
            print("✗ AD Authentication failed.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.unbind()

# Example usage
if __name__ == "__main__":
    print("LDAP Authentication Demo\n")
    
    # Choose demo type
    choice = input("Choose demo: (1) Simple LDAP, (2) Active Directory [1/2]: ").strip()
    
    if choice == "1":
        # Example for OpenLDAP or similar
        ldap_host = input("LDAP server host (e.g., ldap.example.com): ") or "ldap.example.com"
        use_ssl = input("Use LDAPS? (y/n): ").lower() == 'y'
        port = 636 if use_ssl else 389
        base_dn = input("Base DN for search (optional): ") or None
        ldap_auth_simple(ldap_host, port=port, use_ssl=use_ssl, base_dn=base_dn)
    
    elif choice == "2":
        # Example for Active Directory
        ad_domain = input("AD domain or DC (e.g., example.com or dc01.example.com): ") or "example.com"
        ldap_auth_ad(ad_domain)
    
    else:
        print("Invalid choice.")